In [1]:
import pandas as pd
import statistics as stats
import csv
import sys
from math import sqrt

In [2]:
yes_no = ['yes', 'no']
stat_list = ['mean', 'median', 'sd', 'hmean', 'hmedian', 'hsd']
file_path = r'/Users/Luxo_Jr/Desktop/School/Fall 2018/dsc-430-python-programming/data/avocado.csv'
special_columns = ['date', 'year', 'type', 'region']

In [3]:
def keep_going():
    while True:
        cont_ = input('Would you like to start over?(yes or no): ')
        if cont_.lower() not in yes_no:
            print('Please input yes or no only')
        elif cont_.lower() == 'no':
            print('')
            print('Thanks, have a nice day!')
            sys.exit(1)
        elif cont_.lower() == 'yes':
            column_select()
            return None

In [4]:
def column_select():
    
    def stat(stat_selection, data, column_title):
        if stat_selection == 'mean':
            try:
                if column_title in special_columns:
                    mean_ = data.groupby(['%s' % column_title])['%s' % column_title].count().mean()
                    print('That is a date or categorical column so a mean is not available, but the mean value count of' + ' ' + column_title + ' ' + 'is' + ' ' + str(mean_))
                elif column_title not in special_columns:    
                    print('The mean of' + ' ' + column_title + ' ' + 'is' + ' ' + str(stats.mean(data)))
            except TypeError:
                mean_ = data.groupby(['%s' % column_title])['%s' % column_title].count().mean()
                print('The mean of' + ' ' + column_title + ' ' + 'is' + ' ' + str(mean_))
            keep_going()
        elif stat_selection == 'hmean':
            try:
                if column_title in special_columns:
                    print('sorry there is no homegrown mean for date or categorical columns.')
                elif column_title not in special_columns:
                    sum_ = 0
                    for value in list(df['%s' %(column_title)]):
                        sum_ += int(value)
                    hmean_ = sum_/len(data)
                    print('The mean of' + ' ' + column_title + ' ' + 'is' + ' ' + str(hmean_))
            except TypeError:
                mean_ = data.groupby(['%s' % column_title])['%s' % column_title].count().mean()
                print('The mean of' + ' ' + column_title + ' ' + 'is' + ' ' + str(mean_))
            keep_going()
        elif stat_selection == 'median':
            median_ = stats.median(data['%s' %(column_title)])
            try:
                if column_title in special_columns:
                    print('The median value of' + ' ' + column_title + ' ' + 'is' + ' ' + str(median_))
                elif column_title not in special_columns:
                    print('The median of' + ' ' + column_title + ' ' + 'is' + ' ' +  str(median_))
            except TypeError:
                median_ = data.groupby(['%s' % column_title])['%s' % column_title].count().median()
                print('The median of' + ' ' + column_title + ' ' + 'is' + ' ' + str(median_))
            keep_going()
        elif stat_selection == 'sd':
            try:
                if column_title in special_columns:
                    print('Sorry, that is a date or categorical column so the standard deviation is not computable...')
                    keep_going()
                elif column_title not in special_columns:
                    print('The stadard deviation of' + ' ' + column_title + ' ' + 'is' + ' ' + str(stats.stdev(data['%s' %(column_title)])))
            except TypeError:
                sd_ = data.groupby(['%s' % column_title])['%s' % column_title].count().std()
                print('The stnadard deviation of' + ' ' + column_title + ' ' + 'is' + ' ' + str(sd_))
            keep_going()
            return None
    
    #Get column titles
    with open (file_path) as file:
        reader = csv.reader(file)
        column_labels = next(reader)
        column_labels = [x.lower() for x in column_labels]
        print('\033[1m' + """Columns""" + '\033[1m')
        print('\n'.join(column_labels))
    while True:
        try:
            column_name = input('Please input a column name from the list above: ')
            column_name = column_name.lower()
            if column_name not in column_labels:
                print('That column does not exist..')
            else:
                while True: 
                    correct_column = input('You have selected' + ' ' + column_name + ' ' + 'is that correct?(yes or no): ')
                    if correct_column.lower() not in yes_no:
                                print('Please input yes or no only...')
                    elif correct_column.lower() == 'no':
                        column_select()
                    elif correct_column.lower() == 'yes':
                        with open(file_path) as file_:
                            file_.readline()
                            data = []
                            for row in file_: 
                                data.append(row.split(',')[column_labels.index('%s' % (column_name))])
                                #data.remove(data[0])
                            df = pd.DataFrame(data, columns = ['%s' %(column_name)])
                        #df.columns = map(str.lower, df.columns)
                        print('')
                        while True:
                            stat_select = input("""Which stat would you like to compute?:                            
1. mean
2. median
3. standard deviation (sd)
4. homegrown mean (hmean)
5. homegrown median (hmedian)
6. homegrown standard deviation (hsd)

Please only input mean, median, sd, hmean, hmedian, or hsd only.""")
                                                
                            if stat_select not in stat_list:
                                print('Please input mean, median, sd, hmean, hmedian, or hsd only...')
                            elif stat_select == 'sd':
                                stat(stat_select, df, column_name)
                            elif stat_select == 'hmean':
                                stat(stat_select, df, column_name)
                            elif stat_select in stat_list:
                                stat(stat_select, df, column_name)
        except ValueError: 
            print('That column does not exist...')
            return None

In [8]:
def memory_less(column_title, index):
    
    def bin_stats(bins_, count_):
        roll_sum = 0
        for i in range(len(bins_)):
            bin_ = bins_[i]
            roll_sum += bins[i]
            perc = bin_ / count
            print('{:8}{:8}{:8}{:8}'.format(i, bin_, perc, roll_sum))
        return bin_, roll_sum, perc
    
    def bin_vals(bins_, count_, min_, max_):
        bins = [0 for i in range(12)]
        step = (max_ - min_) / (len(bins_) - 2)
        with open(file_path) as file:
            reader = csv.reader(file, delimiter = ',')
            next(reader, None)
            row = next(reader, None)
            while row:
                value = row[index]
                if float(value) < min_:
                    bins[0] += 1
                if float(value) > max_:
                    bins[-1] += 1
                for i in range(len(bins) - 2):
                    lower_bound = min_ + step * i
                    upper_bound = min_ + step * (i + 1)
                    if (float(value) > lower_bound) and (float(value) < upper_bound):
                        bins[i + 1] += 1
                row = next(reader, None)
        return bins
    
    def min_max(bins_, count, min_, max_):
        sum_ = 0
        step = (max_ - min_) / (len(bins_) - 2)
        for i in range (len(bins_)):
            sum_ += bins_[i]
            perc = sum_ / count
            if (perc > 0.5):
                return (min_ + step * (i- 1)), (min_ + step * i)
            
    def candidates(bins_):
        sum_ = 0
        for i in range(1, len(bins_) - 1):
            sum_ += bins_[i]
        return sum_
    
    mstat_select = input("""Which stat would you like to compute?
    1. mean
    2. median
    3. standard deviation (sd)
   
    Please input mean median or sd only: """)
    if mstat_select not in stat_list:
        print('')
        print('Please input mean, median,  or sd only...')
        print('')
    elif mstat_select.lower() == 'mean':
        print('The mean of' + ' ' + column_title + ' ' + 'is' + ' ' + str(mmean(column_title, index)))
    elif mstat_select.lower() == 'sd':
        if column_title in special_columns:
            print('Sorry, that is a date or categorical column so the standard deviation is not computable...')
            print('')
            keep_going()
        else:
            sum_sqs = 0
            count = 0
            x_bar = mmean(column_title, index)
            with open(file_path) as file:
                    reader = csv.reader(file, delimiter = ',')
                    next(reader, None)
                    for row in reader:
                        sum_sqs += (float(row[index]) - x_bar) ** 2
                        count += 1
            var_x = sum_sqs / count - 1
            x_sd = sqrt(var_x)
            print('The stadard deviation of' + ' ' + column_title + ' ' + 'is' + ' ' + str(x_sd))
    elif mstat_select.lower() == 'median':
        
        if column_title in special_columns:
            print('Sorry, that is a date or categorical column so the median is not computable for memoryless...')
            print('')
            keep_going()
        else:
            min_ = 0
            max_ = 0
            count = 0
            bins = [0 for i in range(12)]
            with open(file_path) as file:
                reader = csv.reader(file, delimiter = ',')
                next(reader, None)
                first_val = next(reader)
                min_ = float(first_val[index])
                max_ = float(first_val[index])
                row = next(reader, None)
                while row:
                    value = float(row[index])
                    if float(value) < min_:
                        min_ = float(value)
                    if float(value) > max_:
                        max_ = float(value)
                    count += 1
                    row = next(reader, None)
            cand_ = count
#             while count > 1:
            print('{:8}{:8}{:8}'.format(count, min_, max_))
            bins = bin_vals(bins, count, min_, max_)
            bin_info = bin_stats(bins, count)
            min_, max_ = min_max(bins, count, min_, max_)
            cand_ = candidates(bins)
            print(bins)
            print(bin_info)
            print(min_, max_)
#                 print('{:8}{:3}{:8}{:8}}'.format(bins, bin_info, min_, max_))

In [9]:
memory_less('total volume', 3)

Which stat would you like to compute?
    1. mean
    2. median
    3. standard deviation (sd)
   
    Please input mean median or sd only:  median


   18248   84.5662505646.52
       0       0     0.0       0
       1   178780.9797238053485313   17878
       2     2010.011014905743095134   18079
       3       0     0.0   18079
       4      110.0006028057869355546   18090
       5      470.0025756247259973697   18137
       6      760.004164839982463832   18213
       7      290.0015892152564664622   18242
       8       30.00016440157825515125   18245
       9       15.480052608505042e-05   18246
      10       20.00010960105217010084   18248
      11       0     0.0   18248
[0, 17878, 201, 0, 11, 47, 76, 29, 3, 1, 2, 0]
(0, 18248, 0.0)
84.56 6250640.756


In [8]:
def column_select():
    
    def stat(stat_selection, data, column_title):
        if stat_selection == 'mean':
            if column_title in special_columns:
                df = pd.DataFrame(data, columns = ['%s' %(column_title)])
                mean_ = df.groupby(['%s' % column_title])['%s' % column_title].count().mean()
                print('That is a date or categorical column so a mean is not available, but the mean value count of' + ' ' + column_title + ' ' + 'is' + ' ' + str(mean_))
            elif column_title not in special_columns:
                data = [float(x) for x in data]
                print('The mean of' + ' ' + column_title + ' ' + 'is' + ' ' + str(stats.mean(data)))
            keep_going()
        elif stat_selection == 'hmean':
            if column_title in special_columns:
                print('Sorry there is no homegrown mean for date or categorical columns.')
            elif column_title not in special_columns:
                sum_ = sum([float(x) for x in data])
                hmean_ = sum_ / len(data)
                print('The mean of' + ' ' + column_title + ' ' + 'is' + ' ' + str(hmean_))
            keep_going()
        elif stat_selection == 'mmean':
            
        if stat_selection == 'median':
            df = pd.DataFrame(data, columns = ['%s' %(column_title)])
            median_ = stats.median(df['%s' %(column_title)])
            try:
                if column_title in special_columns:
                    print('The median value of' + ' ' + column_title + ' ' + 'is' + ' ' + str(median_))
                    keep_going()
                elif column_title not in special_columns:
                    print('The median of' + ' ' + column_title + ' ' + 'is' + ' ' +  str(median_))
                    keep_going()
            except TypeError:
                median_ = data.groupby(['%s' % column_title])['%s' % column_title].count().median()
                print('The median of' + ' ' + column_title + ' ' + 'is' + ' ' + str(median_))
                keep_going()
        elif stat_selection == 'hmedian':
            sorted_ = sorted(data)
            index_ = int((len(data) - 1) / 2)
            if column_title in special_columns:
                if len(data) % 2 == 0:
                    hmedian_ = sorted_[index_]
                    print('The median value of' + ' ' + column_title + ' ' + 'is' + ' ' + str(hmedian_))
                    keep_going()
                else:
                    hmedian_1 = sorted_[index_] 
                    hmedian_2 = sorted_[index_ + 1]
                    if hmedian_1 == hmedian_2:
                        print('The median value of' + ' ' + column_title + ' ' + 'is' + ' ' + str(hmedian_1))
                    else:
                        print('The median values of' + ' ' + column_title + ' ' + 'are' + ' ' + str(hmedian_1) + ' ' + 'and' + ' ' + str(hmedian_2))
                    keep_going()
            elif column_title not in special_columns:
                if len(data) % 2 == 1:
                    hmedian_ = sorted_[index_]
                    print('The median value of' + ' ' + column_title + ' ' + 'is' + ' ' + str(hmedian_))
                    keep_going()
                else:
                    hmedian_ = int(float(sorted_[index_])) + int(float(sorted_[index_ + 1])) / 2
                    print('The median value of' + ' ' + column_title + ' ' + 'is' + ' ' + str(hmedian_))
                    keep_going()
        if stat_selection == 'sd':
            if column_title in special_columns:
                print('Sorry, that is a date or categorical column so the standard deviation is not computable...')
                keep_going()
            elif column_title not in special_columns:
                data = [float(x) for x in data]
                print('The stadard deviation of' + ' ' + column_title + ' ' + 'is' + ' ' + str(stats.stdev(data)))
            keep_going()
        elif stat_selection == 'hsd':
            if column_title in special_columns:
                print('Sorry, that is a date or categorical column so the standard deviation is not computable...')
                print('')
                keep_going()
            else:
                data = [float(x) for x in data]
                x_bar = sum([float(x) for x in data]) / len(data)
                var_x = sum([(float(x) - x_bar) ** 2 for x in data]) / len(data)
                x_sd = sqrt(var_x)
                print('The stadard deviation of' + ' ' + column_title + ' ' + 'is' + ' ' + str(x_sd))
                print('')
                keep_going()
            return None
    
    #Get column titles
    with open (file_path) as file:
        reader = csv.reader(file)
        column_labels = next(reader)
        column_labels = [x.lower() for x in column_labels]
        print('\033[1m' + """Columns""" + '\033[1m')
        print('\n'.join(column_labels))
    while True:
        column_name = input('Please input a column name from the list above: ')
        column_name = column_name.lower()
        if column_name not in column_labels:
            print('That column does not exist..')
        else:
            while True: 
                correct_column = input('You have selected' + ' ' + column_name + ' ' + 'is that correct?(yes or no): ')
                if correct_column.lower() not in yes_no:
                            print('Please input yes or no only...')
                elif correct_column.lower() == 'no':
                    column_select()
                elif correct_column.lower() == 'yes':
                    with open(file_path) as file_:
                        file_.readline()
                        data = []
                        for row in file_: 
                            data.append(row.split(',')[column_labels.index('%s' % (column_name))])
                    print('')
                    while True:
                        stat_select = input("""Which stat would you like to compute?:                            
1. mean
2. median
3. standard deviation (sd)
4. homegrown mean (hmean)
5. homegrown median (hmedian)
6. homegrown standard deviation (hsd)

Please only input mean, median, sd, hmean, hmedian, or hsd only.""")

                        if stat_select not in stat_list:
                            print('')
                            print('Please input mean, median, sd, hmean, hmedian, or hsd only')
                        elif stat_select in stat_list:
                            stat(stat_select, data, column_name)

In [9]:
def stat(stat_selection, data, column_title):
    if stat_selection == 'mean':
        if column_title in special_columns:
            df = pd.DataFrame(data, columns = ['%s' %(column_title)])
            mean_ = df.groupby(['%s' % column_title])['%s' % column_title].count().mean()
            print('That is a date or categorical column so a mean is not available, but the mean value count of' + ' ' + column_title + ' ' + 'is' + ' ' + str(mean_))
        elif column_title not in special_columns:
            data = [float(x) for x in data]
            print('The mean of' + ' ' + column_title + ' ' + 'is' + ' ' + str(stats.mean(data)))
        keep_going()
    elif stat_selection == 'hmean':
        if column_title in special_columns:
            print('Sorry there is no homegrown mean for date or categorical columns.')
        elif column_title not in special_columns:
            sum_ = sum([float(x) for x in data])
            hmean_ = sum_ / len(data)
            print('The mean of' + ' ' + column_title + ' ' + 'is' + ' ' + str(hmean_))
        keep_going()
    if stat_selection == 'median':
        df = pd.DataFrame(data, columns = ['%s' %(column_title)])
        median_ = stats.median(df['%s' %(column_title)])
        try:
            if column_title in special_columns:
                print('The median value of' + ' ' + column_title + ' ' + 'is' + ' ' + str(median_))
                keep_going()
            elif column_title not in special_columns:
                print('The median of' + ' ' + column_title + ' ' + 'is' + ' ' +  str(median_))
                keep_going()
        except TypeError:
            median_ = data.groupby(['%s' % column_title])['%s' % column_title].count().median()
            print('The median of' + ' ' + column_title + ' ' + 'is' + ' ' + str(median_))
            keep_going()
    elif stat_selection == 'hmedian':
        sorted_ = sorted(data)
        index_ = int((len(data) - 1) / 2)
        if column_title in special_columns:
            if len(data) % 2 == 0:
                hmedian_ = sorted_[index_]
                print('The median value of' + ' ' + column_title + ' ' + 'is' + ' ' + str(hmedian_))
                keep_going()
            else:
                hmedian_1 = sorted_[index_] 
                hmedian_2 = sorted_[index_ + 1]
                if hmedian_1 == hmedian_2:
                    print('The median value of' + ' ' + column_title + ' ' + 'is' + ' ' + str(hmedian_1))
                else:
                    print('The median values of' + ' ' + column_title + ' ' + 'are' + ' ' + str(hmedian_1) + ' ' + 'and' + ' ' + str(hmedian_2))
                keep_going()
        elif column_title not in special_columns:
            if len(data) % 2 == 1:
                hmedian_ = sorted_[index_]
                print('The median value of' + ' ' + column_title + ' ' + 'is' + ' ' + str(hmedian_))
                keep_going()
            else:
                hmedian_ = int(float(sorted_[index_])) + int(float(sorted_[index_ + 1])) / 2
                print('The median value of' + ' ' + column_title + ' ' + 'is' + ' ' + str(hmedian_))
                keep_going()
    if stat_selection == 'sd':
        if column_title in special_columns:
            print('Sorry, that is a date or categorical column so the standard deviation is not computable...')
            keep_going()
        elif column_title not in special_columns:
            data = [float(x) for x in data]
            print('The stadard deviation of' + ' ' + column_title + ' ' + 'is' + ' ' + str(stats.stdev(data)))
        keep_going()
    elif stat_selection == 'hsd':
        if column_title in special_columns:
            print('Sorry, that is a date or categorical column so the standard deviation is not computable...')
            print('')
            keep_going()
        else:
            data = [float(x) for x in data]
            x_bar = sum([float(x) for x in data]) / len(data)
            var_x = sum([(float(x) - x_bar) ** 2 for x in data]) / len(data)
            x_sd = sqrt(var_x)
            print('The stadard deviation of' + ' ' + column_title + ' ' + 'is' + ' ' + str(x_sd))
            print('')
            keep_going()
        return None

Columns
id
date
averageprice
total volume
4046
4225
4770
total bags
small bags
large bags
xlarge bags
type
year
region


Please input a column name from the list above:  total volume
You have selected total volume is that correct?(yes or no):  yes


Which stat would you like to compute?:                            
1. mean
2. median
3. standard deviation (sd)
4. homegrown mean (hmean)
5. homegrown median (hmedian)
6. homegrown standard deviation (hsd)

Please only input mean, median, sd, hmean, hmedian, or hsd only. sd


The stadard deviation of total volume is 3453545.3553994712


Would you like to start over?(yes or no):  yes


...

The stadard deviation of total volume is 3453450.731237387



Would you like to start over?(yes or no):  no



Thanks, have a nice day!


SystemExit: 1

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
